# Bag of Words und Naive Bayes mit Scikit-Learn


## Sentiment Analysis

basierend auf [diesem](https://sites.pitt.edu/~naraehan/presentation/Movie%20Reviews%20sentiment%20analysis%20with%20Scikit-Learn.html) und [diesem](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#working-with-text-data) und [diesem](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) Tutorial.

In [ ]:
#!pip install scikit-learn
#!pip install nltk
#!pip install matplotlib
#!pip install pandas

In [ ]:
import nltk
import sklearn
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
import pandas as pd

## Bag of Words

Unser Beispiel-Korpus enthält 4 Dokumente:

In [ ]:
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
]

Die Klasse `CountVectorizer` erstellt ein *Bag of Words* aus den Dokumenten. Wir erzeugen aus der Klasse zunächst ein Objekt, das wir `vectorizer` nennen.

In [ ]:
vectorizer = CountVectorizer()

Bei der folgenden Methode `fit_transform()` passieren zwei Schritte:
1. `fit()` : es wird an die Daten "angepasst"
2. `transform()`: die Daten werden entsprechend transformiert

(Diese Methdoen kann man auch separat aufrufen, mit `fit_transform()` ist es aber effizienter). Der Output ist eine *sparse matrix*, das ist eine Form von Matrix, die darauf optimiert wurde, eine Matrix, die hauptsächlich Nullen enthält, effizienter zu repräsentieren.

In [ ]:
# wir speichern das hier nur in einer Variablen um später nochmal "reinschauen" zu können
X = vectorizer.fit_transform(corpus)
X

Wir können uns aber auch die "echte" Matrix (die nicht sparse ist) anschauen:

In [ ]:
X.toarray()

Was sehen wir da? Jede Zeile entspricht einem Dokument. Jede Spalte entspricht einem Feature. Und was unsere Features sind, können wir uns folgendermaßen ausgeben lassen:

In [ ]:
vectorizer.get_feature_names_out()

Nun visualisieren wir das ganze noch etwas hübscher:

In [ ]:
df = pd.DataFrame(X.toarray(),
                 columns = vectorizer.get_feature_names_out(), 
                 index = corpus)

df

Jeder *Worttype* wird also intern auf einen Index abgebildet. Das können wir uns mit dem Attribut `vocabulary_`ausgeben lassen.

In [ ]:
vectorizer.vocabulary_

Um aus einem neuen Dokument (z.B. Testdaten) die entsprechenden Features zu extrahieren, rufen wir die Methode `transform()`(ohne `fit()`!) auf:

In [ ]:
vectorizer.transform(['And another document.']).toarray()

Besteht ein neues Dokument ausschließlich aus Wörtern, die in den Trainingsdaten nicht vorkamen, erhalten wir einen Nullvektor:

In [ ]:
vectorizer.transform(['Something completely new.']).toarray()

### N-Gramme
Statt einem *Bag of Words* mit Unigrammen, können auch N-Gramme beliebiger Größe extrahiert werden. Dazu wird in `CountVectorizer` der Parameter `ngram_range` gesetzt. Das Tupel `(1,2)` bedeutet dass Uni- bis Bigramme extrahiert werden.

In [ ]:
bigram_vectorizer = CountVectorizer(ngram_range=(1,2))
bigram_vectorizer.fit_transform(corpus)
print(bigram_vectorizer.get_feature_names_out())

### Character N-Gramme
Es können auch N-Gramme auf Zeichenbasis (characters) extrahiert werden. Dazu wird der Parameter `analyzer` mit dem Wert `char` oder `char_wb` belegt. Letzeres erstellt N-Gramme nur innerhalb der Wortgrenzen (mit Spaces als Padding)

In [ ]:
character_bigram_vectorizer = CountVectorizer(ngram_range=(1,2), analyzer='char_wb')
character_bigram_vectorizer.fit_transform(corpus)
print(character_bigram_vectorizer.get_feature_names_out())

## Sentiment Analysis for Movie Reviews with Naive Bayes and Bag of Words

Mit der Funktion [`load_files()`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html) können Datensätze in einem bestimmten Format sehr einfach eingelesen werden: Alle Daten, die zu einer Klasse (z. B. *positiv* vs. *negativ*) gehören, müssen in einem eigenen Unterordner liegen. Das ist z.B. beim nltk_movie_reviews Datensatz der Fall (Quelle: https://www.nltk.org/nltk_data/).

In [ ]:
movie = load_files("nltk_movie_reviews")

In [ ]:
# Datenstruktur betrachten

# movie.data : Liste der Reviews
# movie.target : Liste der zugehörigen Klassen (0 vs. 1)
# movie.target_names : Liste aller Klassen

print("Text:", movie.data[0][:100], "...")
print("Klasse:", movie.target[0])
print("Mögliche Klassen:")
for i, klasse in enumerate(movie.target_names):
    print(i, ":", klasse)

### Daten in Trainings- und Testdaten aufsplitten
mit `test_size` (alternativ `train_size`) geben wir an, welcher Anteil der Daten als Testdaten (bzw. Trainingsdaten) verwendet werden soll. Um die absolute Anzahl der Datenpunkte anzugeben, einfach einen Integer-Wert verwenden. Mit `shuffle = True` werden die Datenpunkte zunächst zufällig durcheinandergewürfelt. Damit wir bei jedem Durchgang dasselbe Ergebnis erhalten, muss unbedingt ein `random_state` gesetzt werden (der Wert ist egal!).

In [ ]:
docs_train, docs_test, y_train, y_test = train_test_split(movie.data, movie.target, shuffle=True,
                                                          test_size = 0.20, random_state = 12)

print("Anzahl Trainingsdaten:", len(docs_train))
print("Anzahl Testdaten:", len(docs_test))

### Features extrahieren
Wir verwenden hier einen einfachen *Bag of Words* mit folgenden zusätzlichen Parametern: Nur die 3.000 häufigsten Wörter (`max_features`) und nur diejenigen, die in mindestens zwei Dokumenten auftreten (`min_df`) werden verwendet.

In [ ]:
movieVzer= CountVectorizer(min_df=2, max_features=3000) # use top 3000 words only.
docs_train_counts = movieVzer.fit_transform(docs_train)

### Klassifikator trainieren
Die Klasse `MultinomialNB()` liefert uns einen Naive Bayes Klassifikator (für die Verwendung von Counts; bei binären Features würden wir einen `BernoulliNB()` verwenden). Wir verwenden ihn hier mit Default-Parametern. `fit` bedeutet nichts anderes als *trainieren*. 

In [ ]:
clf = MultinomialNB()
clf.fit(docs_train_counts, y_train)

### Klassifikator anwenden

Aus den Testdaten müssen zunächst dieselben Features extrahiert werden wie aus den Trainingsdaten. Dazu wird der Bag of Words Vectorizer, der auf die Trainingsdaten angepasst wurde, nun auf die Testdaten angewandt (mit `transform()` ohne `fit()`).

In [ ]:
docs_test_counts = movieVzer.transform(docs_test)

Mit `predict()` werden dann die Klassen für die Testdaten vorhergesagt.

In [ ]:
y_pred = clf.predict(docs_test_counts)
y_pred

### Evaluation
Wir vergleichen nun `y_pred`, also die vorhergesagten Klassen, mit `y_test`, also den tatsächlichen Klassen für die Testdaten.

In [ ]:
# Accuracy
accuracy_score(y_test, y_pred)

In [ ]:
# Konfusionsmatrix
cm = confusion_matrix(y_test, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm)
    
cm_display.plot()

In [ ]:
# Klassifikations-Report
print(classification_report(y_test, y_pred))